In [24]:
import cv2
# import numpy as np

# # Load the regular image and mask image
# regular_image_path = r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\regular_images\83.jpg'
# mask_image_path = r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\masked_images\83.jpg'

# regular_image = cv2.imread(regular_image_path, cv2.IMREAD_COLOR)
# mask_image = cv2.imread(mask_image_path, cv2.IMREAD_GRAYSCALE)

# # Create a 5x5 structuring element
# kernel = np.ones((5, 5), np.uint8)

# # Apply dilation to the mask image with 10 iterations
# dilated_mask = cv2.dilate(mask_image, kernel, iterations=20)

# # Save the dilated mask image
# cv2.imwrite(r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\testing_outputs\dilated.png', dilated_mask)

True

In [25]:
# # Apply the mask to the regular image
# masked_image = cv2.bitwise_and(regular_image, regular_image, mask=mask_image)
# dialated_image = cv2.bitwise_and(regular_image, regular_image, mask=dilated_mask)
# cv2.imwrite(r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\testing_outputs\masked_image.png', masked_image)
# cv2.imwrite(r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\testing_outputs\dilated_image.png', dialated_image)

# # Initialize the SIFT feature
# sift = cv2.SIFT_create(contrastThreshold=0.01)

# keypoints_m, descriptors_m = sift.detectAndCompute(masked_image, None)
# result_image_masked = cv2.drawKeypoints(regular_image, keypoints_m, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# keypoints_d, descriptors_d = sift.detectAndCompute(dialated_image, None)
# result_image_dialated = cv2.drawKeypoints(regular_image, keypoints_d, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)


# # Save the image with the SIFT keypoints as a JPG file
# output_image_path = r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\testing_outputs\output_masked.jpg'
# cv2.imwrite(output_image_path, result_image_masked)

# # Save the image with the SIFT keypoints as a JPG file
# output_image_path = r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\testing_outputs\output_dialated.jpg'
# cv2.imwrite(output_image_path, result_image_dialated)

True

In [29]:
from typing import Tuple
import cv2 as OpenCV
import numpy as np

def data_feature_matching_no_cross(image1, image2, descriptors_1, descriptors_2, keypoints_1, keypoints_2) -> np.ndarray:
    matcher = OpenCV.BFMatcher(crossCheck=False)
    feature_matching_output = matcher.match(descriptors_1, descriptors_2)
    matched_image = cv2.drawMatches(image1, keypoints_1,
                                    image2, keypoints_2,
                                    feature_matching_output, None,
                                    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    return matched_image

def data_feature_matching_with_cross(image1, image2, descriptors_1, descriptors_2, keypoints_1, keypoints_2) -> np.ndarray:
    matcher = OpenCV.BFMatcher(crossCheck=True)
    feature_matching_output = matcher.match(descriptors_1, descriptors_2)
    matched_image = cv2.drawMatches(image1, keypoints_1,
                                    image2, keypoints_2,
                                    feature_matching_output, None,
                                    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    #Ransac
    src_pts = np.float32([keypoints_1[m.queryIdx].pt for m in feature_matching_output]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints_2[m.trainIdx].pt for m in feature_matching_output]).reshape(-1, 1, 2)

    _, mask = OpenCV.findHomography(src_pts, dst_pts, OpenCV.RANSAC, 150)
    matches_mask = mask.ravel().tolist()
    ransac_feature_matching = [m for m, keep in zip(feature_matching_output, matches_mask) if keep]
    return cv2.drawMatches(image1, keypoints_1,
                                    image2, keypoints_2,
                                    ransac_feature_matching, None,
                                    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# def apply_ransac(matches, keypoints1, keypoints2, threshold = 3.0, **kwargs):
#     src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
#     dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

#     _, mask = OpenCV.findHomography(src_pts, dst_pts, OpenCV.RANSAC, threshold)
#     matches_mask = mask.ravel().tolist()
#     return [m for m, keep in zip(matches, matches_mask) if keep]

Manual Feature Matching (Mando and Joe)

In [27]:
kernel = np.ones((5, 5), np.uint8)
image1_path = r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\regular_images\1.jpg'
image2_path = r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\regular_images\2.jpg'
masked_image_1_path = r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\masked_images\1.jpg'
masked_image_2_path = r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\masked_images\2.jpg'

image1 = cv2.imread(image1_path, cv2.IMREAD_COLOR)
image2 = cv2.imread(image2_path, cv2.IMREAD_COLOR)
masked_image_1 = cv2.imread(masked_image_1_path, cv2.IMREAD_GRAYSCALE)
masked_image_2 = cv2.imread(masked_image_2_path, cv2.IMREAD_GRAYSCALE)

sift = cv2.SIFT_create(contrastThreshold=0.01)
dilated_mask_1 = cv2.dilate(masked_image_1, kernel, iterations=20)
dilated_mask_2 = cv2.dilate(masked_image_2, kernel, iterations=20)

dialated_image_1 = cv2.bitwise_and(image1, image1, mask=dilated_mask_1)
dialated_image_2 = cv2.bitwise_and(image2, image2, mask=dilated_mask_2)

keypoints_1, descriptors_1 = sift.detectAndCompute(dialated_image_1, None)
result_image1_dialated = cv2.drawKeypoints(image1, keypoints_1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

keypoints_2, descriptors_2 = sift.detectAndCompute(dialated_image_2, None)
result_image2_dialated = cv2.drawKeypoints(image2, keypoints_2, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

In [ ]:
def draw_matches(matcher, image1, image2, keypoints_1, keypoints_2) -> None:
        combined_image = OpenCV.hconcat([
            self.image_one.rgb_image,
            self.image_two.rgb_image
        ])

        for match in matcher:
            x1, y1 = keypoints_1[match.queryIdx].pt
            x2, y2 = keypoints_2[match.trainIdx].pt
            # Draw a line connecting the matched keypoints
            OpenCV.line(
                combined_image, 
                (int(x1), int(y1)), 
                (int(x2) + self.image_one.rgb_image.shape[1], int(y2)), 
                (0, 255, 0), 
                1
            )

        OpenCV.imwrite(output_filename, combined_image)

In [ ]:
matcher = OpenCV.BFMatcher()
matcher.match(descriptors_1, descriptors_2)


In [30]:
# Example usage of the data_feature_matching function
matching_image_no_cross = data_feature_matching_no_cross(result_image1_dialated, result_image2_dialated, descriptors_1, descriptors_2, keypoints_1, keypoints_2)
matching_image_with_cross = data_feature_matching_with_cross(result_image1_dialated, result_image2_dialated, descriptors_1, descriptors_2, keypoints_1, keypoints_2)

# ransac_output = apply_ransac(matcher.match(descriptors_1, descriptors_2), keypoints_1, keypoints_2, threshold=150)

cv2.imwrite(r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\testing_outputs\feature_matches_no_cross.png', matching_image_no_cross)
cv2.imwrite(r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\testing_outputs\ransac_feature_matches.png', matching_image_with_cross)
# cv2.imwrite(r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\testing_outputs\ransac_output.png', ransac_output)

True

In [31]:
print(len(matching_image_no_cross))
print(len(matching_image_with_cross))

3648
3648
